In [1]:
from transformers import AutoTokenizer, AutoModel
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
import torch
import torch.nn as nn

In [3]:
from soft_embedding import SoftEmbedding

n_tokens = 20
initialize_from_vocab = True

# Get the T5 LM-Adapt model

In [4]:
# tokenizer = AutoTokenizer.from_pretrained('google/t5-base-lm-adapt')
# model = AutoModel.from_pretrained('google/t5-base-lm-adapt')

tokenizer = T5Tokenizer.from_pretrained("google/t5-base-lm-adapt")
model = T5ForConditionalGeneration.from_pretrained("google/t5-base-lm-adapt")

## Set the prompt embeddings

In [5]:
model.get_input_embeddings()

Embedding(32128, 768)

In [6]:
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)

In [7]:
model.encoder.set_input_embeddings(s_wte)

In [8]:
model.encoder.get_input_embeddings()

SoftEmbedding(
  (wte): Embedding(32128, 768)
)

In [9]:
model.decoder.get_input_embeddings()

Embedding(32128, 768)

In [10]:
# freeze all parameters except the soft embedding
for n, p in model.named_parameters():
    if "learned_embedding" not in n:
        print("freezing", n)
        p.requires_grad = False

freezing shared.weight
freezing encoder.block.0.layer.0.SelfAttention.q.weight
freezing encoder.block.0.layer.0.SelfAttention.k.weight
freezing encoder.block.0.layer.0.SelfAttention.v.weight
freezing encoder.block.0.layer.0.SelfAttention.o.weight
freezing encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
freezing encoder.block.0.layer.0.layer_norm.weight
freezing encoder.block.0.layer.1.DenseReluDense.wi_0.weight
freezing encoder.block.0.layer.1.DenseReluDense.wi_1.weight
freezing encoder.block.0.layer.1.DenseReluDense.wo.weight
freezing encoder.block.0.layer.1.layer_norm.weight
freezing encoder.block.1.layer.0.SelfAttention.q.weight
freezing encoder.block.1.layer.0.SelfAttention.k.weight
freezing encoder.block.1.layer.0.SelfAttention.v.weight
freezing encoder.block.1.layer.0.SelfAttention.o.weight
freezing encoder.block.1.layer.0.layer_norm.weight
freezing encoder.block.1.layer.1.DenseReluDense.wi_0.weight
freezing encoder.block.1.layer.1.DenseReluDense.wi_1.weight


## Test out an input

In [11]:
inputs = tokenizer("Potatoes are very", return_tensors="pt")

inputs.input_ids.shape, inputs.attention_mask.shape

(torch.Size([1, 6]), torch.Size([1, 6]))

In [12]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [13]:
# from easydict import EasyDict

In [14]:
# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
model_inputs = {}
model_inputs['labels'] = inputs['input_ids']
model_inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
model_inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

print(model_inputs['input_ids'].shape, model_inputs['attention_mask'].shape)
outputs = model(**model_inputs)
# outputs = model.generate(**model_inputs)

torch.Size([1, 26]) torch.Size([1, 26])


In [15]:
outputs.loss

tensor(4.0335, grad_fn=<NllLossBackward>)

In [16]:
outputs.logits

tensor([[[-21.8414,  -3.9433,  -4.4774,  ..., -23.1774, -20.8651, -23.2463],
         [-28.9616,  -7.3654,  -7.0188,  ..., -28.7094, -29.3238, -28.3979],
         [-55.6495, -18.5982, -19.4930,  ..., -55.2821, -56.4373, -55.4934],
         [-29.2188,  -7.6311,  -6.6200,  ..., -29.0802, -29.6543, -28.9161],
         [-28.1646,  -7.5980,  -8.9950,  ..., -28.0778, -28.5367, -27.8298],
         [-28.1100,  -6.5290,  -9.9915,  ..., -27.9909, -28.5121, -27.7928]]],
       grad_fn=<UnsafeViewBackward>)

In [17]:
outputs = model.generate(model_inputs['input_ids'], max_length=20)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'good for you.'

# Train on SQuAD

In [18]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Found cached dataset squad (/home/kzl6/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [19]:
squad[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [20]:
# inputs = tokenizer(squad[0]['question'], squad[0]['context'], return_tensors="pt")
inputs = tokenizer(squad[0]['context'], return_tensors="pt")
inputs.keys(), inputs['input_ids'].shape

(dict_keys(['input_ids', 'attention_mask']), torch.Size([1, 180]))

In [21]:
def preprocess_function(examples, n_tokens=20):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt"
    )
    
    answers = [a['text'][0].strip() for a in examples["answers"]]
    targets = tokenizer(
        answers,
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt"
    )
        
    model_inputs['input_ids'] = torch.cat([torch.full((inputs['input_ids'].shape[0],n_tokens), 50256), 
                                           inputs['input_ids']], 1)
    model_inputs['attention_mask'] = torch.cat([torch.full((inputs['input_ids'].shape[0],n_tokens), 1), 
                                                inputs['attention_mask']], 1)
    model_inputs['labels'] = targets['input_ids']
    
    return model_inputs

In [22]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad.column_names)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [23]:
tokenized_squad.set_format("torch")

In [24]:
tokenized_squad[0]['input_ids'].shape

torch.Size([404])

In [25]:
from tqdm.notebook import tqdm
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 1
num_training_steps = num_epochs * len(tokenized_squad)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

train_dataloader = DataLoader(tokenized_squad, shuffle=True, batch_size=8)

In [26]:
model.train()
model.to('cuda')

for epoch in tqdm(range(num_epochs)):
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# inputs = tokenizer(context + question, return_tensors="pt")

# answer_inputs = tokenizer(answer, return_tensors="pt")

# model_inputs = {}
# model_inputs['decoder_input_ids'] = answer_inputs['input_ids']
# model_inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
# model_inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)


In [28]:
squad_test = load_dataset("squad", split="validation[:5]")
len(squad_test), squad_test[0]

Found cached dataset squad (/home/kzl6/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


(5,
 {'id': '56be4db0acb8001400a502ec',
  'title': 'Super_Bowl_50',
  'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
  'question': 'Which NFL team represented the AFC at Super Bowl 50?',
  'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Br

In [29]:
model.eval()

for test_qa in squad_test:
    inputs = tokenizer(
        test_qa["question"],
        test_qa["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt"
    )
    model_inputs['input_ids'] = torch.cat([torch.full((inputs['input_ids'].shape[0],n_tokens), 50256), inputs['input_ids']], 1).cuda()
    model_inputs['attention_mask'] = torch.cat([torch.full((inputs['input_ids'].shape[0],n_tokens), 1), inputs['attention_mask']], 1).cuda()
    with torch.no_grad():
        outputs = model.generate(model_inputs['input_ids'], max_length=20)
        pred_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Question:", test_qa["question"])
    print("Predicted:", pred_answer)
    print("True:", test_qa["answers"]["text"])
    print("*******")

Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted: The NFL's Super Bowl 50 logo was a re-imagined version of
True: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
*******
Question: Which NFL team represented the NFC at Super Bowl 50?
Predicted: The NFL's Super Bowl 50 logo was a re-imagined version of
True: ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']
*******
Question: Where did Super Bowl 50 take place?
Predicted: The NFL's Super Bowl 50 logo was a re-imagined version of
True: ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]
*******
Question: Which NFL team won Super Bowl 50?
Predicted: The NFL's Super Bowl 50 logo was a re-imagined version of
True: ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
*******
Question: What color was used to emphasize the 50th anniversary of the Super Bowl?
Predicted: The NFL's logo was a re-branding of the Super Bowl, which
True

# Finetune on Transfer Task